In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score,precision_score,f1_score,recall_score
# Models
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,VotingRegressor,BaggingRegressor,RandomForestRegressor,RandomTreesEmbedding
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from catboost import CatBoost,CatBoostRegressor
from xgboost import XGBRegressor,XGBRFRegressor
from flaml import AutoML
# Other
import pickle
import wandb

PROJECT_NAME = 'House-Prices-Advanced-Regression-Techniques-V12'
device = 'cuda'

In [2]:
torch.manual_seed(469)
np.random.seed(469)
random.seed(469)

In [3]:
data = pd.read_csv('./data/train.csv')

In [4]:
def valid(model,X,y,valid=False):
    preds = model.predict(X)
    if valid:
        results = {
            'val mean_absolute_error':mean_absolute_error(y_true=y,y_pred=preds),
            'val mean_squared_error':mean_squared_error(y_true=y,y_pred=preds),
        }
    else:
        results = {
            'mean_absolute_error':mean_absolute_error(y_true=y,y_pred=preds),
            'mean_squared_error':mean_squared_error(y_true=y,y_pred=preds),
        }
    return results

In [5]:
def fe(data,col):
    max_num = data[col].quantile(0.95)
    min_num = data[col].quantile(0.05)
    data = data[data[col] > max_num]
    data = data[data[col] < min_num]
    return data

In [6]:
def object_to_int(data,col):
    data_col = data[col].to_dict()
    idx = -1
    labels_and_int_index = {}
    for data_col_vals in data_col.values():
        if data_col_vals not in labels_and_int_index.keys():
            idx += 1
            labels_and_int_index[data_col_vals] = idx
    new_data = []
    for data_col_vals in data_col.values():
        new_data.append(labels_and_int_index[data_col_vals])
    data[col] = new_data
    return data,idx,labels_and_int_index,new_data

In [7]:
def make_submission(model,name):
    data = pd.read_csv('./data/test.csv')
    ids = data['Id']
    for col,missing in zip(list(data.columns),data.isna().sum()):
        if missing > 0:
            try:
                data[col] = data[col].fillna(data[col].median())
            except:
                i = data[col].value_counts().to_dict()
                data[col] = data[col].fillna(i[list(i.keys())[0]])
    one_hot_cols = []
    for col,dtype in zip(list(data.columns),data.dtypes):
        if dtype == object:
            try:
                data[col].astype(float)
            except:
                one_hot_cols.append(col)
    data = mct.transform(data.astype(str))
    data = data.astype(float)
    data = data.toarray()
    preds = model.predict(data)
    df = pd.DataFrame({'Id':ids,'SalePrice':preds})
    df.to_csv(f'./submissions/{name}.csv',index=False)

In [8]:
def train(model,X_train,X_test,y_train,y_test,name):
    wandb.init(project=PROJECT_NAME,name=name)
    model.fit(X_train,y_train)
    wandb.log(valid(model,X_train,y_train,True))
    wandb.log(valid(model,X_train,y_train,False))
    make_submission(model,name)
    return model

In [9]:
for col,dtype,num_of_missing in zip(list(data.columns),data.dtypes,data.isna().sum()):
    if dtype == float or dtype == int:
        data[col] = data[col].fillna(data[col].median())
    else:
        data[col] = data[col].fillna(0)
        data,idx,labels_and_int_index,new_data = object_to_int(data,col)
        data[col].replace(0,data[col].median())

In [10]:
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,0,65.0,8450,0,0,0,0,0,...,0,0,0,0,0,2,2008,0,0,208500
1,2,20,0,80.0,9600,0,0,0,0,0,...,0,0,0,0,0,5,2007,0,0,181500
2,3,60,0,68.0,11250,0,0,1,0,0,...,0,0,0,0,0,9,2008,0,0,223500
3,4,70,0,60.0,9550,0,0,1,0,0,...,0,0,0,0,0,2,2006,0,1,140000
4,5,60,0,84.0,14260,0,0,1,0,0,...,0,0,0,0,0,12,2008,0,0,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,0,62.0,7917,0,0,0,0,0,...,0,0,0,0,0,8,2007,0,0,175000
1456,1457,20,0,85.0,13175,0,0,0,0,0,...,0,0,1,0,0,2,2010,0,0,210000
1457,1458,70,0,66.0,9042,0,0,0,0,0,...,0,0,3,1,2500,5,2010,0,0,266500
1458,1459,20,0,68.0,9717,0,0,0,0,0,...,0,0,0,0,0,4,2010,0,0,142125


In [12]:
# for col,dtype,num_of_missing in zip(list(data.columns),data.dtypes,data.isna().sum()):
#     print(col,dtype,num_of_missing)

In [13]:
X = data.drop('SalePrice',axis=1)

In [14]:
y = data['SalePrice']

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.0625)